OECD API with Python
======

## Example showing GDP growth rates by selected country

-----

*September 3, 2017*<br>
*@bd_econ*

The documentation for the Organization for Economic Cooperation and Development (OECD) API can be found [here](https://data.oecd.org/api/)

In [1]:
import requests
import pandas as pd

## Parameters/ Settings

In [2]:
base = 'http://stats.oecd.org/sdmx-json/data/'
param = [('dataset', 'QNA'),
         ('country', 'FRA+ITA+ESP+GBR'),
         ('indicators', 'GDP+B1_GE.CUR+VOBARSA'), 
         ('freq', 'Q'), 
         ('start_date' , '?startTime=1999-Q4')
        ]

series = '.'.join(x[1] for x in param[1:-1])
url = '{}{}/{}{}'.format(base, param[0][1], series, param[-1][1])

## Request data

In [3]:
r = requests.get(url).json()

In [5]:
date_list = r['structure']['dimensions']['observation'][0]['values']
dates = pd.to_datetime([x['id'] for x in date_list])
    
areas = [v['name'] for v in r['structure']['dimensions']['series'][0]['values']]

title = r['structure']['dimensions']['series'][1]['values'][0]['name']

df = pd.DataFrame()
for i, area in enumerate(areas):
    s_key = '{}:0:0:0'.format(i)
    s_list = r['dataSets'][0]['series'][s_key]['observations']
    df[area] = pd.Series([s_list[val][0] for val in sorted(s_list, key=int)])
    df[area] = (((df[area]/df[area].shift())**4)-1)*100
df.index = dates
df = df.dropna()
print(title)
df.tail().round(1)

Gross domestic product - expenditure approach


,France,Italy,Spain,United Kingdom
2016-04-01,-0.4,0.4,3.4,2.4
2016-07-01,0.7,1.2,2.8,2.0
2016-10-01,2.1,1.6,2.8,2.7
2017-01-01,2.0,1.8,3.2,0.9
2017-04-01,1.9,1.5,3.5,1.2
